In [9]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

SHORT_URL = "https://eodhistoricaldata.com/api/shorts/{sym}.{exc}?api_token={token}&from={from_date}"
EOD_URL = "https://eodhistoricaldata.com/api/eod/{sym}.{exc}?api_token={token}&from={from_date}&to={to_date}"
API_KEY = "5f243129807d34.64565602"
DATA_DIR = Path('/Users/eric/Projects/stonks/ml/rnn/data/')
SYMBOLS = ['SPY', 'QQQ', 'VOO', 'GLD', 'EEM', 'IEMG', 'VTI', 'IVV', 'VEA', 'VTV', 'GDX', 'EFA', 'XLF']

def to_api_date(date: datetime) -> str:
    return date.strftime("%Y-%m-%d")

def short_interest(sym="AAPL", exc="US", token="OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX", days=365*5) -> pd.DataFrame:
    from_date = datetime.now() - timedelta(days=days)
    url = SHORT_URL.format(sym=sym, exc=exc, token=token, from_date=to_api_date(from_date))
    print(url)
    df = pd.read_csv(url, skipfooter=1)
    return df

def eod(start_date:datetime, sym="AAPL", exc="US", token="OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX") -> pd.DataFrame:
    to_date = start_date + timedelta(days=100)
    print(f"From: {start_date}, To: {to_date}")
    now = datetime.now()
    df = None
    while start_date < now:
        url = EOD_URL.format(sym=sym, exc=exc, token=token, from_date=to_api_date(start_date), to_date=to_api_date(to_date))
        print(url)
        _df = pd.read_csv(url, skipfooter=1)
        if df is None:
            df = _df
        else:
            df = pd.concat((df, _df))
        start_date += timedelta(days=100)
        to_date += timedelta(days=100)

    return df

In [10]:
def save_with_short(sym:str, exc:str, token:str)->pd.DataFrame:
    short_df = short_interest(sym, token=API_KEY).tail(100)
    short_interest(sym, exc=exc, token=token).tail(100)
    start_date = pd.to_datetime(short_df['Date'].iloc[0], infer_datetime_format=True)
    start_date: datetime = start_date.to_pydatetime()
    eod_df = eod(start_date, sym, token=API_KEY)
    short_df = short_df.set_index("Date")
    short_df = short_df.join(eod_df["Volume"], lsuffix="_eod")
    short_df["%short"] = short_df["Volume"] / short_df["Volume_eod"]
    df = eod_df.join(short_df, on="Date", lsuffix="_short").reset_index()
    df.ffill(inplace=True)
    df.drop(["Volume_eod"], axis=1, inplace=True)
    df.to_csv(DATA_DIR / f"chart_{sym}.csv")
    return df

In [ ]:
for s in SYMBOLS:
    save_with_short(s, "US", API_KEY)

https://eodhistoricaldata.com/api/shorts/AAPL.US?api_token=5f243129807d34.64565602&from=2015-08-02
https://eodhistoricaldata.com/api/shorts/SPY.US?api_token=5f243129807d34.64565602&from=2015-08-02
From: 2018-09-14 00:00:00, To: 2018-12-23 00:00:00
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2018-09-14&to=2018-12-23
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2018-12-23&to=2019-04-02
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2019-04-02&to=2019-07-11
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2019-07-11&to=2019-10-19
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2019-10-19&to=2020-01-27
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2020-01-27&to=2020-05-06
https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=5f243129807d34.64565602&from=2020-05

/Users/eric/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/Users/eric/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:30: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
